## Virtual file system support

Pyolite mounts a virtual `/drive` directory which allows access to files that are
provided by JupyterLab.

**NOTE:** The virtual file system **does not work when in private mode under Firefox**
and you'll get permission errors. It will also fail to work if you **hard refresh** the
page.

In [ ]:
import os

In [ ]:
os.getcwd()

### Make directory

In [ ]:
os.path.isdir("dir")

In [ ]:
os.mkdir("dir")

### List directory content

In [ ]:
os.listdir(os.getcwd())

### Remove

In [ ]:
os.rmdir("dir")

In [ ]:
os.listdir(os.getcwd())

### Reading/writing file

In [ ]:
with open("test.txt", "w") as fobj:
    fobj.write("Hello! I write this from the kernel worker")

In [ ]:
with open("test.txt", "r") as fobj:
    print(fobj.read())

In [ ]:
from PIL import Image

img = Image.new("RGB", (60, 30), color=(73, 109, 137))
img.save("pil_color.png")

In [ ]:
from IPython.display import Image

with open("pil_color.png", "rb") as fobj:
    data = fobj.read()

print(data[:20])
Image(data=data, format="png")

### Replace

In [ ]:
os.path.exists("test.txt")

In [ ]:
os.rename("test.txt", "test_rename.txt")

In [ ]:
os.path.exists("test.txt")

In [ ]:
os.listdir(os.getcwd())